In [3]:
import pandas as pd
from scipy.fft import fft, ifft, fftfreq
import scipy.signal as sng
import numpy as np
import matplotlib.pyplot as plt
import numpy as np

In [801]:
eeg=pd.read_csv('извлеченные_ээг/eeg_S01.csv')

In [802]:
#electrodes=np.delete(eeg.columns, [0, 1, -1, -2, -3])
electrodes=['P3', 'C3', 'F3', 'Fz', 'F4', 'C4', 'P4', 'Cz', 'Pz', 'Fp1', 'Fp2', 'T3', 'T5', 'O1', 'O2', 'F7', 'F8', 'T6', 'T4']
fs=300 #частота дискретизации ээг

In [803]:
#Почему-то данные в виде строки, так что нужно перегнать обратно в числа#
def THE_MOST_STUPID_FUNCTION(eeg_producta):
    list_eeg={}
    for i in electrodes:
        eeg_producta_i=np.delete(np.array(eeg_producta[i].split('[')), [0, 1])
        s=[]
        for j in range(len(eeg_producta_i)):
            vrem=eeg_producta_i[j].split(',')[:-1]
            vrem[-1]=vrem[-1][:-1]
            for k in range(len(vrem)):
                vrem[k]=float(vrem[k])
            s.append(vrem)
        list_eeg[f'{i}']=s
    return list_eeg

In [804]:
#Выделим частоты и спектральные плотности мощности (psd)#
def get_welt(eeg_producta, fs=300):
    product_elecrod_psd={}
    #electrodes=['P3', 'C3', 'F3', 'Fz', 'F4', 'C4', 'P4', 'Cz', 'Pz', 'Fp1', 'Fp2', 'T3', 'T5', 'O1', 'O2', 'F7', 'F8', 'T6', 'T4']
    electrodes=['F3', 'F4']
    for i in electrodes:
        eeg_i=eeg_producta[i]
        s=[]
        for j in eeg_i:
            welt=sng.welch(j, fs=fs)
            freq=welt[0]
            psd=welt[1]
            alpha_freq=freq[freq>8]
            alpha_freq=alpha_freq[alpha_freq<15]
            if len(alpha_freq)!=0:
                start=np.where(freq==alpha_freq[0])
                stop=np.where(freq==alpha_freq[-1])
                alpha_psd=psd[start[0][0]:stop[0][0]+1]
                s.append({'freq_alpha_diapazon': alpha_freq, 'psd': alpha_psd})
        product_elecrod_psd[f'{i}']=s
    return product_elecrod_psd

In [805]:
#То же, но для всех продуктов#
def get_all_psd(eeg):
    s={}
    for i in range(len(eeg)):
        eeg_producta=eeg.loc[i]
        krivie_ruki=THE_MOST_STUPID_FUNCTION(eeg_producta=eeg_producta)
        pp=eeg['product_page'].loc[i]
        pn=eeg['product_number'].loc[i]
        
        eeg_psded=get_welt(krivie_ruki)
        s[f'{pp} {pn}']=[eeg['index'].loc[i], eeg['info'].loc[i], eeg_psded]
    return s

In [ ]:
evrika=get_all_psd(eeg=eeg)

In [807]:
#Усредним все psd для каждого продукта как в каждом временном отрезке наблюдения, так и по всем ним#

#Я НЕ ВЫКИДЫВАЛ ВРЕМЕННЫЕ ОТРЕЗКИ НАБЛЮДЕНИЯ КОНКРЕТНОГО ПРОДУКТА, КОТОРЫЕ ЯВНО КОРОЧЕ ОСТАЛЬНЫХ#

def averager(eeg_pesded):
    for i in eeg_pesded:
        eeg_page_prod=eeg_pesded[i][2]
        electrodes=['F3', 'F4']
        for k in electrodes:
            vrem=[]
            for j in range(len(eeg_page_prod[k])):
                evrika[i][2][k][j][f'mean_psd_{k}']=np.mean(eeg_page_prod[k][j]['psd'])
                vrem.append(np.mean(eeg_page_prod[k][j]['psd']))
            evrika[i][2][f'{k}_average_all_of_saw']=np.mean(vrem)
averager(evrika)

c:\Users\Егор\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\fromnumeric.py:3504: RuntimeWarning: Mean of empty slice.
  return _methods._mean(a, axis=axis, dtype=dtype,
c:\Users\Егор\AppData\Local\Programs\Python\Python311\Lib\site-packages\numpy\core\_methods.py:129: RuntimeWarning: invalid value encountered in scalar divide
  ret = ret.dtype.type(ret / rcount)


In [808]:
#Посчитаем фронтальную альфа-ассиметрию (aw) для каждого продукта#
def get_AW(eeg_avereged):
    s=[]
    inf=[]
    for i in eeg_avereged:
        mean_page_prod=eeg_avereged[i][2]
        aw_prod=float((mean_page_prod['F4_average_all_of_saw']-mean_page_prod['F3_average_all_of_saw'])/(mean_page_prod['F4_average_all_of_saw']+mean_page_prod['F3_average_all_of_saw']))
        s.append(aw_prod)
        inf.append(int(eeg_avereged[i][1][11]))
    return [s, inf]

In [809]:
hell=get_AW(evrika)

In [810]:
df=pd.DataFrame({'product_page_number': list(evrika)})
df['aw']=hell[0]
df['buy']=hell[1]

In [811]:
df.dropna(inplace=True)
df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 129 entries, 0 to 131
Data columns (total 3 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   product_page_number  129 non-null    object 
 1   aw                   129 non-null    float64
 2   buy                  129 non-null    int64  
dtypes: float64(1), int64(1), object(1)
memory usage: 4.0+ KB


In [812]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics

In [813]:
x=df.drop(columns=['product_page_number', 'buy'])
y=df.drop(columns=['product_page_number', 'aw'])

In [814]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.20, random_state=10)

In [815]:
#Посмотрим, может быть и получилось#
LR=LogisticRegression()
LR.fit(x_train, y_train)
y_predict_LR=LR.predict(x_test)
print(metrics.accuracy_score(y_test, y_predict_LR))

0.9230769230769231


c:\Users\Егор\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [ ]:
#Сделаем для всех испытуемых#
people=['01', '02', '03', '05', '06', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']
electrodes=['P3', 'C3', 'F3', 'Fz', 'F4', 'C4', 'P4', 'Cz', 'Pz', 'Fp1', 'Fp2', 'T3', 'T5', 'O1', 'O2', 'F7', 'F8', 'T6', 'T4']
fs=300 #частота дискретизации ээг
df=pd.DataFrame()

for t in people:
        eeg=pd.read_csv(f'извлеченные_ээг/eeg_S{t}.csv')
        evrika=get_all_psd(eeg=eeg)
        averager(evrika)
        hell=get_AW(evrika)
        df_people=pd.DataFrame({'people': np.full(len(list(evrika)), t), 'product_page_number': list(evrika)})
        df_people['aw']=hell[0]
        df_people['buy']=hell[1]
        df=pd.concat([df, df_people])

In [ ]:
#def aw_God(directory='извлеченные_ээг'):
#    people=['01', '02', '03', '05', '06', '07', '08', '09', '10', '12', '13', '14', '15', '16', '17', '18', '19', '20', '21', '22', '23', '24', '25', '26', '27', '28', '29', '30', '31', '32', '33', '34', '35', '36', '37', '38', '39', '40', '41', '42', '43', '44']
#    electrodes=['P3', 'C3', 'F3', 'Fz', 'F4', 'C4', 'P4', 'Cz', 'Pz', 'Fp1', 'Fp2', 'T3', 'T5', 'O1', 'O2', 'F7', 'F8', 'T6', 'T4']
#    fs=300 #частота дискретизации ээг
#    df=pd.DataFrame()
#    for t in people:
#        eeg=pd.read_csv(f'{directory}/eeg_S{t}.csv')
#        evrika=get_all_psd(eeg=eeg)
#        averager(evrika)
#        hell=get_AW(evrika)
#        df_people=pd.DataFrame({'people': np.full(len(list(evrika)), t), 'product_page_number': list(evrika)})
#        df_people['aw']=hell[0]
#        df_people['buy']=hell[1]
#        df=pd.concat(df, df_people)
#    return df

In [820]:
df

,people,product_page_number,aw,buy
0,01,Page1 0,-0.044624,0
1,01,Page1 1,0.156658,0
2,01,Page1 2,0.340922,0
3,01,Page1 3,-0.139705,1
4,01,Page1 4,0.125743,0
...,...,...,...,...
137,44,Page6 19,-0.128140,1
138,44,Page6 20,-0.023829,0
139,44,Page6 21,0.088120,0
140,44,Page6 22,0.334975,0


In [821]:
#df.to_csv('eeg_aw.csv')

In [8]:
df=pd.read_csv('eeg_aw.csv')
df.dropna(inplace=True)

In [9]:
#Посмотрим, может быть и получилось#
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn import metrics
x=df.drop(columns=['product_page_number', 'buy', 'Unnamed: 0', 'people'])
y=df.drop(columns=['product_page_number', 'aw', 'Unnamed: 0', 'people'])

In [18]:
x_train, x_test, y_train, y_test = train_test_split(x, y, test_size=0.10, random_state=10)

In [19]:
LR=LogisticRegression()
LR.fit(x_train, y_train)
y_predict_LR=LR.predict(x_test)
print(metrics.accuracy_score(y_test, y_predict_LR))

0.8671328671328671


c:\Users\Егор\AppData\Local\Programs\Python\Python311\Lib\site-packages\sklearn\utils\validation.py:1300: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
